In [8]:
import math
import numpy as np
import pandas as pd
import tensorflow as tf

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from mpl_toolkits.axes_grid1 import make_axes_locatable

from scipy import ndimage, misc
import datetime
from datetime import timedelta

import keras
from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential, load_model
from keras.layers import Dense, Flatten, Reshape, Conv1D, MaxPooling1D, AveragePooling1D,\
UpSampling1D, InputLayer

from scipy import ndimage, misc

from sklearn import svm
from sklearn.ensemble import IsolationForest
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, auc
from sklearn.model_selection import cross_val_score, train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import MaxAbsScaler
import joblib
from sklearn.neighbors import LocalOutlierFactor
from sklearn import cluster, datasets

In [14]:
root_path = "/Users/I559573/Downloads/SoundSamples"
path_fan = root_path + "/fan/train/"
path_pump = root_path + "/pump/train/"
path_valve = root_path + "/valve/train/"
path_gearbox = root_path + "/gearbox/train/"
print(path_fan)
print(path_pump)
print(path_valve)
print(path_gearbox)

/Users/I559573/Downloads/SoundSamples/fan/train/
/Users/I559573/Downloads/SoundSamples/pump/train/
/Users/I559573/Downloads/SoundSamples/valve/train/
/Users/I559573/Downloads/SoundSamples/gearbox/train/


In [13]:
fan_data = pd.DataFrame()
pump_data = pd.DataFrame()
gearbox_data = pd.DataFrame()
valve_data = pd.DataFrame()

In [259]:
fan_data = pd.read_csv(root_path + '/fan.csv', sep="_", header=None, nrows=100000, 
                       names=["a", "sec", "c", "source", "status", "id", "g", "wind_strength", "i", "l", "m", "n"])

pump_data = pd.read_csv(root_path + '/pump.csv', sep="_", header=None, nrows=100000, 
                        names=["a", "sec", "c", "source", "status", "id", "g", "h", "serial_no", "l", "m", "n"])

valve_data = pd.read_csv(root_path + '/valve.csv', sep="_", header=None, nrows=100000, 
                         names=["a", "sec", "c", "source", "status", "id", "g", "pattern", "i", "l", "m", "n"])

gearbox_data = pd.read_csv(root_path + '/gearbox_2.csv', sep="_", header=None, nrows=100000,
                       names=["a", "sec", "c", "source", "status", "id", "weight", "u_weight", "lenght", "u_lenght", "voltage", "u_voltage", "o"])

In [260]:
fan_data = fan_data.replace(np.nan, "N/A")
pump_data = pump_data.replace(np.nan, "N/A")
gearbox_data = gearbox_data.replace(np.nan, "N/A")
valve_data = valve_data.replace(np.nan, "N/A")

fan_data[["i", "l", "m", "n"]] = fan_data[["i", "l", "m", "n"]].astype(str)
pump_data[["l", "m", "n"]] = pump_data[["l", "m", "n"]].astype(str)
valve_data[["i", "l", "m", "n"]] = valve_data[["i", "l", "m", "n"]].astype(str)

In [269]:
fan_data.head()

,sec,source,status,id,wind_strength,attributes
0,0,fan,normal,0,1,ambient
1,0,fan,normal,1,1,ambient
2,0,fan,normal,2,1,ambient
3,0,fan,normal,3,1,ambient
4,0,fan,normal,4,1,ambient


In [270]:
pump_data.head()

,sec,source,status,id,h,serial_no,attributes
0,0,pump,normal,0,no,000,water
1,0,pump,normal,1,no,000,water
2,0,pump,normal,2,no,000,water
3,0,pump,normal,3,no,000,water
4,0,pump,normal,4,no,000,water


In [271]:
valve_data.head()

,sec,source,status,id,pattern,attributes
0,0,valve,normal,0,0,no pump
1,0,valve,normal,1,0,no pump
2,0,valve,normal,2,1,air pump
3,0,valve,normal,3,1,air pump
4,0,valve,normal,4,0,air pump


In [272]:
gearbox_data.head()

,sec,source,status,id,weight,u_weight,lenght,u_lenght,voltage,u_voltage,attributes
0,0,gearbox,normal,0000,0.0,g,25.0,mm,2000.0,mV,none
1,0,gearbox,normal,0001,0.0,g,25.0,mm,2000.0,mV,none
2,0,gearbox,normal,0002,0.0,g,25.0,mm,1000.0,mV,none
3,0,gearbox,normal,0003,100.0,g,25.0,mm,2000.0,mV,none
4,0,gearbox,normal,0004,250.0,g,25.0,mm,2000.0,mV,none


In [261]:
fan_data = fan_data.assign(source='fan')
fan_data['attributes'] = fan_data[["i", "l", "m", "n"]].agg('_'.join, axis=1)

pump_data = pump_data.assign(source='pump')
pump_data['attributes'] = pump_data[["l", "m", "n"]].agg('_'.join, axis=1)

valve_data = valve_data.assign(source='valve')
valve_data['attributes'] = valve_data[["i", "l", "m", "n"]].agg('_'.join, axis=1)

gearbox_data = gearbox_data.assign(source='gearbox')
gearbox_data['attributes'] = gearbox_data["o"]

In [262]:
fan_data['attributes'] = fan_data['attributes'].replace({'_N/A':''},regex=True)
fan_data = fan_data.replace({'.wav':''},regex=True)
fan_data = fan_data.replace({'N/A':''},regex=True)
fan_data = fan_data.replace({'_':' '},regex=True)

pump_data['attributes'] = pump_data['attributes'].replace({'_N/A':''},regex=True)
pump_data = pump_data.replace({'.wav':''},regex=True)
pump_data = pump_data.replace({'N/A':''},regex=True)
pump_data = pump_data.replace({'_':' '},regex=True)

valve_data['attributes'] = valve_data['attributes'].replace({'_N/A':''},regex=True)
valve_data = valve_data.replace({'.wav':''},regex=True)
valve_data = valve_data.replace({'N/A':''},regex=True)
valve_data = valve_data.replace({'_':' '},regex=True)

gearbox_data['attributes'] = gearbox_data['attributes'].replace({'_N/A':''},regex=True)
gearbox_data = gearbox_data.replace({'.wav':''},regex=True)
gearbox_data = gearbox_data.replace({'N/A':''},regex=True)
gearbox_data = gearbox_data.replace({'_':' '},regex=True)

In [273]:
fan_data['attributes'].unique()

array(['ambient', 'big ambient', 'small ambient', 'Fact D ambient'],
      dtype=object)

In [274]:
pump_data['attributes'].unique()

array(['water', 'viscous liquid', '', '021'], dtype=object)

In [275]:
pump_data['serial_no'].unique()

array(['000', '010', '021', '020'], dtype=object)

In [276]:
valve_data['attributes'].unique()

array(['no pump', 'air pump', 'air pump comb'], dtype=object)

In [277]:
gearbox_data['attributes'].unique()

array(['none', 'None', ''], dtype=object)

In [268]:
fan_data = fan_data.drop(["a", "c", "g", "i", "l", "m", "n"], axis=1)
pump_data = pump_data.drop(["a", "c", "g", "l", "m", "n"], axis=1)
valve_data = valve_data.drop(["a", "c", "g", "i", "l", "m", "n"], axis=1)
gearbox_data = gearbox_data.drop(["a", "c", "o"], axis=1)